In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj 
import os
import shutil
import PIL
import xarray as xr
import glob
from functools import partial
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [14]:
sourcepath = '/home/mor582/seagrass/TH'

In [15]:
def get_longitude(item):
    longitude =float(item[0]) + float(item[2][0:-1])/60 + float(item[3][0:-1])/3600
    return (longitude)

def convert_wgs_to_utm(lon, lat):
    utm_band = str((np.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code


def process_transect(leg):
    leg = leg.copy()
    leg['GPSTime'] = leg.index
    leg['GPSDistance'] =((leg['Northing'].diff()**2 + leg['Easting'].diff()**2)**0.5)
    leg['GpsSpeed']=(((leg['Northing'].diff()**2 + leg['Easting'].diff()**2)**0.5)/leg.GPSTime.diff().dt.total_seconds())
    leg['timedelta'] = leg.GPSTime.diff().dt.total_seconds()
    leg['GPSExcessTime'] = ((leg['GpsSpeed']/leg['DroneSpeed']) * leg['timedelta'])-leg['timedelta']
    leg['ImageTime'] = leg.index
    leg['ImageInterval']=leg['ImageTime'].diff().dt.total_seconds()
    leg.loc[leg['ImageInterval']==2,'ImageTime'] =leg.loc[leg['ImageInterval']==2,'ImageTime'] + pd.to_timedelta('500L')
    leg['ImageInterval']=leg['ImageTime'].diff().dt.total_seconds()
    for i in range(1,len(leg)):
        leg.iloc[i,leg.columns.get_loc('GPSTime')] = leg.iloc[i,leg.columns.get_loc('GPSTime')] + pd.to_timedelta(leg.iloc[i,leg.columns.get_loc('GPSExcessTime')],unit='s')
        leg['timedelta'] = leg.GPSTime.diff().dt.total_seconds()
        leg['GpsSpeed']=(((leg['Northing'].diff()**2 + leg['Easting'].diff()**2)**0.5)/leg.GPSTime.diff().dt.total_seconds())
        leg['GPSExcessTime'] = ((leg['GpsSpeed']/leg['DroneSpeed']) * leg['timedelta'])-leg['timedelta']
    gpstime=leg.GPSTime.astype(int).astype("float")
    imagetime= leg.ImageTime.astype('int64').astype('float')
    leg['ImageNorthing'] =np.interp(imagetime,gpstime,leg.Northing)
    leg['ImageEasting'] = np.interp(imagetime,gpstime,leg.Easting)
    leg['ImageDistance'] =((leg['ImageNorthing'].diff()**2 + leg['ImageEasting'].diff()**2)**0.5)
    leg['ImageSpeed'] =leg['ImageDistance']/leg['ImageInterval']
    return leg

In [37]:
def correct_flight(file):
    output = file.replace('_RENAME','_IMG')
    if not os.path.exists(output):
        data =pd.read_csv(file,index_col='Time',parse_dates=['Time','DateTimeOriginal'])
        data.sort_index(inplace=True)
        data['Hem']=np.nan
        data.loc[data['GPSLatitudeRef']=='North','Hem']=1
        data.loc[data['GPSLatitudeRef']=='South','Hem']=-1
        # Drop all pictures with No GPS
        data = data[(~data['GPSLongitude'].isna() | ~data['GPSLatitude'].isna())]
        data['Longitude'] = np.nan
        data['Latitude'] = np.nan
        data['Longitude']=data['GPSLongitude'].str.split(' ',expand=True).apply(get_longitude,axis=1)
        data['Latitude']=data['GPSLatitude'].str.split(' ',expand=True).apply(get_longitude,axis=1) * data['Hem']
        utmcode =convert_wgs_to_utm(data['Longitude'][0],data['Latitude'][0])
        utmproj =Proj(init='epsg:{0:1.5}'.format(utmcode),datum='WGS84',ellps='WGS84')
        e,n =utmproj(data['Longitude'].values,data['Latitude'].values)
        data['Easting']=e
        data['Northing']=n
        data['Crs'] = 'epsg:{0:1.5}'.format(utmcode)

        data['LocalTime']=data.index
        data['Interval']=pd.to_datetime(data.LocalTime).diff().dt.total_seconds()


        # data.loc[data['Interval']==4,'LocalTime'] =data.loc[data['Interval']==4,'LocalTime'] - pd.to_timedelta('500L')
        # data['Interval']=pd.to_datetime(data.LocalTime).diff().dt.total_seconds()
        data['GpsDist']=(data['Northing'].diff()**2 + data['Easting'].diff()**2)**0.5
        data['GpsSpeed']=((data['Northing'].diff()**2 + data['Easting'].diff()**2)**0.5)/data['Interval']
        data['DroneSpeed'] = (data['SpeedX']**2+data['SpeedY']**2)**0.5

        data['Leg'] =0
        data.loc[data['Interval']>8,'Leg'] =1
        data['Leg'] = data['Leg'].cumsum()
        data['Counter'] = 1
        data['Counter'] = data['Counter'].cumsum()
        data.index[data.index.duplicated()]+pd.to_timedelta('500L')
        data =data.sort_values('SourceFile')
        idx =data.index.duplicated('last')
        data =data[~idx]
        g = data.groupby('Leg')
        corrected =pd.concat([process_transect(leg) for name,leg in g])
        fig =plt.figure()
        ax = plt.subplot()
        corrected.plot(ax=ax,y='GpsSpeed',ylim=[0,12])
        corrected.plot(ax=ax,y='Interval',ylim=[0,12])
        corrected.plot(ax=ax,y='DroneSpeed',ylim=[0,12])
        corrected.plot(ax=ax,y='ImageDistance',ylim=[0,30])
        corrected.plot(ax=ax,y='ImageSpeed',ylim=[0,35])
        corrected =corrected.rename({'Northing':'OldNorthing','Easting':'OldEasting'})
        corrected['Northing']= corrected['ImageNorthing']
        corrected['Easting']= corrected['ImageEasting']
        corrected.index.name='Time'
        corrected.to_csv(output,index=True)

In [41]:
processfiles = glob.glob(sourcepath+'/**/*_RENAME.csv',recursive=True)
for pfile in processfiles:
    print(pfile)
    correct_flight(pfile)


/home/mor582/seagrass/TH/KMK02_001/DJIP4_TH_KMK02_001_RENAME.csv
/home/mor582/seagrass/TH/KMK01_001/DJIP4_TH_KMK01_001_RENAME.csv


In [40]:
processfiles

['/home/mor582/seagrass/TH/KMK02_001/DJIP4_TH_KMK02_001_RENAME.csv',
 '/home/mor582/seagrass/TH/KMK01_001/DJIP4_TH_KMK01_001_RENAME.csv']